# pyDOV demo

## Import van biblioteken

We zullen eerst de Pandas bibliotheek importeren die wordt gebruikt voor om te gaan met data in tabellen. Een Pandas ``DataFrame`` kan gezien worden als een tabel. We zullen het aantal rijen een kolommen dat in de notebook wordt getoond aanpassen.

In [ ]:
import pandas as pd
pd.options.display.max_columns = 200
pd.options.display.max_rows = 1000

Vervolgens kunnen we de nodige functionaliteit importeren uit ``pydov``. We kunnen de klasse ``SonderingSearch`` importeren om naar CPTs te zoeken en de klasses ``Within`` en ``Box`` om binnen een rechthoek naar punten te zoeken.

In [ ]:
from pydov.search.sondering import SonderingSearch
from pydov.util.location import Within, Box

We zullen ook Plotly importeren om grafieken te maken:

In [ ]:
from plotly import tools, subplots
import plotly.express as px
import plotly.graph_objs as go
import plotly.io as pio
import plotly.figure_factory as ff
from plotly.colors import DEFAULT_PLOTLY_COLORS
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode()
pio.templates.default = 'plotly_white'
pio.templates['plotly'].layout['autosize'] = False
for key in pio.templates.keys():
    pio.templates[key].layout['autosize'] = False

## Opzoeken van CPTs

Om sonderingen op te zoeken starten we met het aanmaken van een ``SonderingSearch`` object. Dit object kan gebruikt worden om de opzoeking te doen, met door de gebruiker gekozen zoekparameters.

We kunnen sonderingen opzoeken binnen een ``Box``, een rechthoek begrensd door een punt linksonder en rechtsboven. Deze coordinaten kunnen uit de DOV verkenner of topografische rapporten gehaald worden. De Lambert 72 projectie wordt gebruikt in Vlaanderen.

Eens de ``Box`` gedefinieerd is, kunnen we de opzoeking starten door de ``.search`` methode toe te passen. Deze methode voert de API call uit en plaatst de resultaten in een Pandas ``DataFrame``. We kennen dit ``DataFrame`` toe aan de variabele ``df``.

In [ ]:
sondering = SonderingSearch()
df = sondering.search(location=Within(Box(103641, 188873, 103700, 189046)))

## Bekijken van de gevonden CPTs

Eens dit is uitgevoerd, kunnen we kijken wat er in ``df`` zit. We kunnen de eerste vijf rijen van de tabel printen met de methode ``.head``.

In [ ]:
df.head()

We kunnen zien dat we per punt van de CPT (dus voor elke diepte $z$, of waarde van de kolom ``lengte``) een rij hebben.

We kunnen ook een statistische samenvatting van de waarden in de verschillende kolommen krijgen met de methode ``.describe``.

In [ ]:
df.describe()

Om er de verschillende CPTs uit te halen, kunnen we kijken naar de unieke waarden van de kolom ``sondeernummer``.

In [ ]:
df['sondeernummer'].unique()

We kunnen ook kijken hoeveel unieke CPTs er zijn met de methode ``.__len__``

In [ ]:
df['sondeernummer'].unique().__len__()

Er zijn 16 CPTs. We kunnen zien welke sondeermethodes er zijn toegepast en dan kunnen we er de CPTs met een elektrische conus uitfilteren. We kijken dan hoeveel dat er zijn:

In [ ]:
df['sondeermethode'].unique()

In [ ]:
df_elektrisch = df[df['sondeermethode'] == 'continu elektrisch']
df_elektrisch['sondeernummer'].unique().__len__()

Er is 1 continu elektrische sondering. We zullen die verder verwerken met ``groundhog``.

In [ ]:
df_elektrisch['sondeernummer'].unique()

In [ ]:
df_elektrisch.head()

## CPT verwerking met ``groundhog``

``groundhog`` is een bibliotheek met grondmechanische functionaliteit, ook voor verwerking van CPTs. Importeren en visualiseren van CPT resultaten gaat heel snel met ``groundhog``. Hier zullen we de data voor CPT ``WD-07/129-S1-TT/MAN2`` importeren en visualiseren.

Eerst kunnen we de klasse ``PCPTProcessing`` importeren. Deze klasse bevat de nodige functionaliteit voor de verwerking.

In [ ]:
from groundhog.siteinvestigation.insitutests.pcpt_processing import PCPTProcessing

De workflow start door een ``PCPTProcessing`` object aan te maken. We geven dit de titel van onze sondering.

In [ ]:
cpt = PCPTProcessing(title='WD-07/129-S1-TT/MAN2')

De ``PCPTProcessing`` klasse in ``groundhog`` bevat een method ``load_pydov`` die rechtstreeks inladen van CPTs adhv een sondeernummer toelaat.

De tabel uit de ``SonderingSearch`` toonde dat de diepte ook soms gegeven is in de kolom ``lengte``. We kunnen dit expliciet opgeven in de methode ``load_pydov``.

In [ ]:
cpt.load_pydov(name='WD-07/129-S1-TT/MAN2', z_key='lengte')

De data is nu ingeladen en het ``.data`` attribuut van ``cpt`` is een ``DataFrame`` (tabel) met de CPT data. De volgende kolommen zijn steeds vereist in ``groundhog`` en hebben gestandardiseerde namen:

   - ``z [m]``: Diepte van de conus
   - ``qc [MPa]``: Conusweerstand
   - ``fs [MPa]``: Mantelwrijving
   - ``u2 [MPa]``: Poriënwateroverspanning (niet vaak uitgevoerd in Vlaanderen)

In [ ]:
cpt.data.head()

Het visualiseren van de CPT data kan met één enkele regel code. ``groundhog`` geeft standaard poriënwateroverspanningen weer. Omdat die hier ontbreken kunnen we ipv poriënwateroverspanningen het wrijvingsgetal $ R_f $ weergeven.

De grafiek is interactief, dus er kan ingezoomd worden op geselecteerde dieptes. Waardes worden ook weergegeven door over de data te <i>hoveren</i>.

In [ ]:
cpt.plot_raw_pcpt(plot_friction_ratio=True)

Ook toepassen van correlaties is eenvoudig.

We kunnen de correlatie tussen CPT data en wrijvingshoek volgens Kulhawy & Mayne (1990) toepassen op deze CPT. De documentatie voor deze correlatie vind je via de link hieronder.

https://groundhog.readthedocs.io/en/main/site_investigation/pcpt_functions.html#groundhog.siteinvestigation.insitutests.pcpt_correlations.frictionangle_sand_kulhawymayne

De formule voor de correlatie is:

$$ \varphi^{\prime} = 17.6 + 11.0 \cdot \log_{10} \left[  \frac{q_t / P_a}{ \sqrt{\sigma_{vo}^{\prime} / P_a}} \right] $$ 

Deze formule is gecalibreerd aan proeven in een calibratiekamer, een drukvat waarin de spanningscondities in de grond worden nagebootst. (http://www.ismgeo.it/en/camera_calibrazione_en.html). Bij toepassen van deze correlaties is het steeds van belang om na te kijken of de zanden in de CPT vergelijkbaar zijn aan de zanden die in de studie met de calibratiekamer werden gebruikt (mineralogie, gehalte aan fijn materiaal <63$\mu$m, ...).

De correlatie maakt gebruikt van de gecorigeerde conusweerstand $ q_t $ maar voor een elektrische conus zonder poriënwaterspanningssensor is die gelijk aan $ q_c $ (zie cursus <i>Geotechnics</i>).

We moeten ook de verticale effectieve spanning berekenen. Als we uitgaan van een effectief volumegewicht van 9kN/m$^3$ kunnen we zie eenvoudig berekenen. Merk op dat de bewerkingen vectorieel zijn, elke waarde van diepte wordt met 9 vermenigvuldigd en toegekend aan de kolom ``'Vertical effective stress [kPa]'``.

In [ ]:
cpt.data['qt [MPa]'] = cpt.data['qc [MPa]']
cpt.data['Vertical effective stress [kPa]'] = 9 * cpt.data['z [m]']
cpt.data.head()

Vervolgens kunnen we de formule van Kulhawy & Mayne toepassen op de volledige CPT. De documentatie toont dat de output van de functie een Python dictionary is met <i>key</i> ``'Phi [deg]'`` voor de wrijvingshoek. We zullen deze <i>mappen</i> naar een kolom met naam ``'Friction angle Kulhawy [deg]'``.

In [ ]:
cpt.apply_correlation('Friction angle Kulhawy and Mayne (1990)', outputs={'Phi [deg]': 'Friction angle Kulhawy [deg]'})

We kunnen de data weergeven na berekening:

In [ ]:
cpt.data.head()

Tenslotte kunnen we een grafiek maken Plotly van de conusweerstand en de berekende wrijvingshoek. We bouwen deze stap voor stap op (zie commentaar).

In [ ]:
# Maak een figuur met twee subplots naast elkaar, de diepte-as wordt gedeeld
fig = subplots.make_subplots(rows=1, cols=2, print_grid=False, shared_yaxes=True)
# Maak een trace aan voor de conusweerstand
trace1 = go.Scatter(x=cpt.data['qc [MPa]'], y=cpt.data['z [m]'], showlegend=True, mode='lines',name='qc')
# Ken deze trace toe aan het linkse panel
fig.append_trace(trace1, 1, 1)
# Maak een trace aan voor de wrijvingshoek
trace2 = go.Scatter(x=cpt.data['Friction angle Kulhawy [deg]'], y=cpt.data['z [m]'], showlegend=True, mode='lines',name='phi')
# Ken deze trace toe aan het rechtse panel
fig.append_trace(trace2, 1, 2)
# Defineer de titel, positie en range van de X-as voor het linkse subplot
fig['layout']['xaxis1'].update(title=r'$ q_c \ \text{[MPa]}$', side='top', anchor='y', range=(0, 50))
# Defineer de titel, positie en range van de X-as voor het rechtse subplot
fig['layout']['xaxis2'].update(title=r'$ \varphi^{\prime} \ \text{[deg]}$', side='top', anchor='y', range=(20, 50))
# Defineer de titel, positie en range van de diepte-as
fig['layout']['yaxis1'].update(title=r'$ z \ \text{[m]}$', autorange='reversed')
# Kies de hoogte en breedte van de grafiek
fig['layout'].update(height=800, width=700)
# Geef de grafiek weer
fig.show()

# BRO demo

We kunnen hetzelfde uitvoeren voor een CPT in Nederland. De syntax is minder <i>clean</i> dan voor pyDOV, maar de procedure werkt wel robuust.

## Import van packages

Eerst kunnen we een aantal packages importeren die nodig zijn voor de verwerking:

In [ ]:
import xml.etree.ElementTree as ET
import requests
import re
import numpy as np

## Opzoeken van sonderingen volgens geografische positie

De basis-URL voor API-calls wordt dan opgegeven.

In [ ]:
url = "https://publiek.broservices.nl/sr/cpt/v1/characteristics/searches?requestReference=request"

We kunnen een verzoek sturen op basis van een zekere registratieperiode en geografische positie (cirkel met straal 500m rond punt met gegeven lengte- en breedtegraad. Dat verzoek sturen we met een ``.post`` request.

In [ ]:
# setup the query params (literal copy of the bro example in the docs)
my_obj = {
    "registrationPeriod": {"beginDate": "2017-01-01", "endDate": "2021-01-01"},
    "area": {
        "enclosingCircle": {
            "center": {"lat": 52.038297852, "lon": 5.31447958948},
            "radius": 0.5,
        }
    },
}

# get it..
x = requests.post(url, json=my_obj)

Dit geeft een hoop XML (eXtensible Markup Language) terug die we kunnen <i>parsen</i>.

## XML parsing

In [ ]:
x.text

De code om de IDs van de CPTs uit de XML te halen staat hieronder. Hier moet niets aan veranderd worden, wat van belang is, zijn de ``broIDs`` die eruit komen. Op basis van deze IDs kunnen we dan verder individuele CPTs ophalen.

In [ ]:
root = ET.fromstring(x.text)

In [ ]:
broIds = []
srids = []
eastings = []
northings = []
for child in root:
    if child.tag == "{http://www.broservices.nl/xsd/dscpt/1.1}dispatchDocument":
        for elem in child:
            for e in elem.findall("{http://www.broservices.nl/xsd/brocommon/3.0}broId"):
                broIds.append(e.text)
            for e in elem.findall("{http://www.broservices.nl/xsd/brocommon/3.0}deliveredLocation"):
                srids.append(e.attrib['srsName'][-5:])
                for _t in e:
                    eastings.append(float(re.split(' ', _t.text)[0]))
                    northings.append(float(re.split(' ', _t.text)[1]))
            
broIds, srids, eastings, northings

## Verwerking van één BRO CPT

We kunnen de CPT met ID ``'CPT000000053406'`` ophalen en verwerken. We gebruiken hiervoor het package geotexxx van de gemeente Amsterdam (credit Thomas Van de Linden).

In [ ]:
from geotexxx.gefxml_reader import Cpt

We sturen onze request naar een specifieke URL die gedetailleerde CPT data teruggeeft.

In [ ]:
detail_url = f"https://publiek.broservices.nl/sr/cpt/v1/objects/%s" % broIds[1]
detail_url

We can send a ``get`` request to the URL to retrieve the CPT data.

In [ ]:
resp = requests.get(detail_url)

The response (``resp``) is XML test format. We can parse this using the ``Cpt`` class in the ``geotexxx`` package. Using the method ``.load_xml``, the response text can be loaded.

In [ ]:
cpt_detail = Cpt()

cpt_detail.load_xml(resp.text, fromFile=False)

The CPT data is contained in the ``data`` attribute. This is a Pandas ``DataFrame`` which we can show. Note that the column headers are different from the ``groundhog`` convention.

In [ ]:
cpt_detail.data.head()

To import this CPT data in ``groundhog``, we will need to declare which column names to use for depth, tip resistance, sleeve friction and pore water pressure. A pore water pressure column is not available so we will first add it to the CPT data and fill it with NaN (not a number) values.

In [ ]:
# Fill all rows in the column 'u2 [MPa]' with the value NaN
cpt_detail.data.loc[:, 'u2 [MPa]'] = np.nan

Now, we can generate a ``groundhog`` ``PCPTProcessing`` object.

In [ ]:
cpt_bro = PCPTProcessing("BRO CPT")

As the CPT data is already available in a Pandas ``DataFrame``, we can use the method ``.load_pandas`` to load the CPT data. This is also where we specify which column name to use for depth (``z_key``), cone tip resistance (``qc_key``) and sleeve friction (``fs_key``). The key for pore pressure (``u2 [MPa]``) is already what ``groundhog`` expects.

In [ ]:
cpt_bro.load_pandas(cpt_detail.data, z_key="penetrationLength", qc_key="coneResistance", fs_key="localFriction")

The CPT is now loaded and can be plotted. Again, friction ratio is plotted instead of pore pressure. As the default range for $ q_c $ is quite large (0 - 100MPa), we can finetune is with the ``qc_range`` argument.

In [ ]:
cpt_bro.plot_raw_pcpt(plot_friction_ratio=True, qc_range=(0, 40))